In [ ]:
#<Ind != a, cnn, EDR, h3c3>
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import numpy as np
import random
import pandas as pd
from utils.utils2 import *
import os
import sklearn
from sklearn.mixture import GaussianMixture
from scipy import stats
from options import Options
opt = Options().parse()
from data_provider.DS import * 

# data prepare
trainX = pd.read_csv('./data_provider/datasets/'+opt.stream_sensor+'.csv', sep='\t')
trainX.columns = ["id", "datetime", "value"] 
trainX.sort_values('datetime', inplace=True),
R_X = pd.read_csv('./data_provider/datasets/'+opt.rain_sensor+'.csv', sep='\t')
R_X.columns = ["id", "datetime", "value"] 
R_X.sort_values('datetime', inplace=True)
ds = DS(opt, trainX, R_X)

In [ ]:
# model training
from models.DAN_M import *
model = DAN(opt,ds)
model.train()

In [ ]:
# inferencing the whole test set for experiments
from models.DAN_M import *
model = DAN(opt,ds)
ds.refresh_dataset(trainX, R_X)
model.model_load()
model.inference()

In [ ]:
# inferencing some selected test data for real-world application
from models.Inference import *

# data prepare
testP=["2025-08-01 06:30:00" ]
indicator_data = [random.uniform(0, 1)/800 for _ in range(288)]
stream_data = [random.uniform(0, 1) for _ in range(15*24*4)]
                 
model = DAN_I(opt)
model.model_load()

for i in range(len(testP)):
    p = model.predict(testP[i], stream_data, indicator_data)
    print(p)

In [ ]:
import pandas as pd
import numpy as np
import math
from metric import *
sensor_id=4996
val_set=pd.read_csv('./data_provider/datasets/test_timestamps_24avg.tsv',sep='\t')
val_points=val_set["Hold Out Start"]
trainX = pd.read_csv('./data_provider/datasets/'+opt.stream_sensor+'.csv', sep='\t')
trainX.columns = ["id", "datetime", "value"] 
count = 0
for test_point in val_points:
    point = trainX[trainX["datetime"]==test_point].index.values[0]
    NN = np.isnan(trainX[point-15*24*4:point+3*24*4]["value"]).any() 
    if not NN:
        count += 1
print(count)
# install predicted values
print("There are totally ", len(val_points), " test points")
filename_bs = 'Saratoga_S_fixed_OS18_Shed-ProbFeatureSaratoga_withRain_pred_lists_print.tsv'
vals4 = []
path4 = './output/Saratoga_withRain/test/'
with open(path4+filename_bs, 'r') as f:
    pred_ext_val=f.readline()
    for i in range(count):
      pred_ext_val=f.readline()
      kk=pred_ext_val.split("\t")
      kk=[ float(tt)  for tt in kk[1:] ]
      vals4.append(kk)
# compute metrics
mean_x=[]
rmse_x=[]
loop=0
all_GT=[]
all_DAN=[]
loop = 0
ind = 0
while loop<len(val_points):
    ii=val_points[loop]
    val_point=val_points[ind]
    point=trainX[trainX["datetime"]==ii].index.values[0]
    x=trainX[point-15*24*4:point+3*24*4]["value"].values.tolist()    
    if (np.isnan(np.array(x)).any()):
        print("For point, ", val_point, ", there is None value in it!")
        loop = loop + 1 # id for time list 
        continue
    loop = loop + 1
    if ind >= count - count%100:
        break
    mean_vals_x=np.array(x[15*24*4:]).mean()
    mean_x.append(mean_vals_x)        
    ind += 1   # id for dataset 
# DAN
    temp_vals4=list([None for i in range(15*24*4)])+list(vals4[ind-1])
    all_GT.extend(x[15*24*4:])
    all_DAN.extend(temp_vals4[15*24*4:]) 
mean = np.array(all_GT).mean()
print("mean is, ", mean)

In [ ]:
metric_g("DAN", np.array(all_DAN), np.array(all_GT))